**Calculate the dice coefficient between MR-based and CT-based simulations**
----
### Context
This notebook shows an example that calculates the dice coefficient between MR-based and CT-based Simulations for the 54 EEG locations. It is modified for the data with SDR = 0.55. This code can also be adapted for different datasets. 

In [ ]:
import pandas as pd
import sys
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import h5py
from BabelViscoFDTD.H5pySimple import ReadFromH5py

## Additions to path
We add the paths to BabelBrain

In [ ]:
sys.path.append('The_BabelBrain_Path')
sys.path.append('The_BabelBrain_Path/BabelBrain')

## Select the EEG locations

In [ ]:
SELECTED_FOR_FUS = pd.read_csv('SELECTED_EEG.csv', index_col='Name')
# Filter rows where the 'Include' column is 'YES'
SELECTED_FOR_FUS = SELECTED_FOR_FUS[SELECTED_FOR_FUS['Include'] == 'YES']['Include'].to_dict()

## Define a function

In [ ]:
# Define a function to compute the Dice coefficient
def dice_coefficient(mask1, mask2):
    intersection = np.logical_and(mask1, mask2)
    return 2.0 * intersection.sum() / (mask1.sum() + mask2.sum())

## Calculating the Dice coefficients

In [ ]:
BasePath='The_path_to_the_folder/SDR_0p55'    #Define the path to the simulation folder
ComputingDevice='M1'
Frequency=500e3       #Selecet the fundamental frequncy
PPW=6
TxSystem='Single'
results = []

for ID in SELECTED_FOR_FUS:
    # Construct the file paths for the two masks
    mask1_filename = f'{ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc50.0_Diam50.0_DataForSim.h5'
    mask2_filename = f'CT_{ID}_{TxSystem}_{int(Frequency/1e3)}kHz_{PPW}PPW_Foc50.0_Diam50.0_DataForSim.h5'
    
    mask1_file_path = os.path.join(BasePath, mask1_filename)
    mask2_file_path = os.path.join(BasePath, mask2_filename)

    if os.path.isfile(mask1_file_path) and os.path.isfile(mask2_file_path):
        mask1_h5 = ReadFromH5py(mask1_file_path)
        mask_charm = np.flip(mask1_h5['MaterialMap'], axis=2)
        mask2_h5 = ReadFromH5py(mask2_file_path)
        mask_ct = np.flip(mask2_h5['MaterialMap'], axis=2)

        SkullRegion_charm = (mask_charm == 2) | (mask_charm == 3)
        SkullRegion_ct = (mask_ct == 2) | (mask_ct == 3)
        
        # Compute the Dice coefficient between the two masks
        dice_coeff = dice_coefficient(SkullRegion_charm, SkullRegion_ct)

        results.append({'ID': ID, 'Dice Coefficient': dice_coeff})
    else:
        print(f'Files not found for {ID}')

df = pd.DataFrame(results)

# Save the DataFrame to an Excel file
df.to_excel('dice_coefficient_results.xlsx', index=False)

